In [7]:
import PIL, random, sys
from PIL import Image, ImageDraw, ImageEnhance
import numpy as np
import json
import os

from scipy.io.wavfile import read

In [16]:
# Set scaling factor for wave form
scaling_factor = 10
# Set scaling factor for dot size
dot_scaling_factors = [100]
# Number of dots
starting_index = 10000
nums_dots = [7500]
# Set image dimensions
width = 1024
height = 1024

palettes = {
    # light blue / pink
    "Blue/Pink" : {0:"#e3f2fd",
         -1:"#bbdefb",
         -2:"#90caf9",
         -3:"#64b5f6",
         -4:"#42a5f5",
         -5:"#2196f3",
         -6:"#1e88e5",
         -7:"#1976d2",
         -8:"#1565c0",
         -9:"#0d47a1",
         -10:"#1a237e",
         1:"#fce4ec",
         2:"#f8bbd0",
         3:"#f48fb1",
         4:"#f06292",
         5:"#ec407a",
         6:"#e91e63",
         7:"#d81b60",
         8:"#c2185b",
         9:"#ad1457",
         10:"#880e4f"},
    # indigo / red
    "Indigo/Red" : {0:"#e8eaf6",
         -1:"#c5cae9",
         -2:"#9fa8da",
         -3:"#7986cb",
         -4:"#5c6bc0",
         -5:"#3f51b5",
         -6:"#3949ab",
         -7:"#303f9f",
         -8:"#283593",
         -9:"#1a237e",
         -10:"#304ffe",
         1:"#ffebee",
         2:"#ffcdd2",
         3:"#ef9a9a",
         4:"#e57373",
         5:"#ef5350",
         6:"#f44336",
         7:"#e53935",
         8:"#d32f2f",
         9:"#c62828",
         10:"#b71c1c"},
    # indigo / amber
    "Indigo/Amber" : {0:"#e8eaf6",
         -1:"#c5cae9",
         -2:"#9fa8da",
         -3:"#7986cb",
         -4:"#5c6bc0",
         -5:"#3f51b5",
         -6:"#3949ab",
         -7:"#303f9f",
         -8:"#283593",
         -9:"#1a237e",
         -10:"#304ffe",
         1:"#fff8e1",
         2:"#ffecb3",
         3:"#ffe082",
         4:"#ffd54f",
         5:"#ffca28",
         6:"#ffc107",
         7:"#ffb300",
         8:"#ffa000",
         9:"#ff8f00",
         10:"#ff6f00"},
    # gradient
    "Gradient" : {0:"#EC4B18",
         -1:"#EC4B18",
         -2:"#EC4B18",
         -3:"#EC4B18",
         -4:"#5C44EC",
         -5:"#5C44EC",
         -6:"#5C44EC",
         -7:"#5C44EC",
         -8:"#4684EF",
         -9:"#4684EF",
         -10:"#4684EF",
         1:"#F384AD",
         2:"#F384AD",
         3:"#F384AD",
         4:"#FDF078",
         5:"#FDF078",
         6:"#FDF078",
         7:"#FDF078",
         8:"#47F4D4",
         9:"#47F4D4",
         10:"#47F4D4"},
    # Purple / Amber
    "Purple/Amber" : {0:"#f3e5f5",
         -1:"#e1bee7",
         -2:"#e1bee7",
         -3:"#ba68c8",
         -4:"#ab47bc",
         -5:"#9c27b0",
         -6:"#8e24aa",
         -7:"#7b1fa2",
         -8:"#6a1b9a",
         -9:"#4a148c",
         -10:"#311b92",
         1:"#fff8e1",
         2:"#ffecb3",
         3:"#ffe082",
         4:"#ffd54f",
         5:"#ffca28",
         6:"#ffc107",
         7:"#ffb300",
         8:"#ffa000",
         9:"#ff8f00",
         10:"#ff6f00"},
    # Rainbow
    "Rainbow" : {0:"#c8e6c9",  #green
         -1:"#a5d6a7", #green
         -2:"#ffecb3", #amber
         -3:"#ffd54f", #amber
         -4:"#ffc107", #amber
         -5:"#ef5350", #orange
         -6:"#f44336", #orange
         -7:"#b71c1c", #orange
         -8:"#ef5350", #red
         -9:"#f44336", #red
         -10:"#b71c1c", #red
         1:"#e8f5e9",  #green
         2:"#bbdefb", #blue
         3:"#64b5f6", #blue
         4:"#2196f3", #blue
         5:"#c5cae9", #indigo
         6:"#7986cb", #indigo
         7:"#3f51b5", #indigo
         8:"#e1bee7", #purple
         9:"#ba68c8", #purple
         10:"#9c27b0"},  #purple
    # Autumnal
    "Autumnal" : {0:"#ffeb3b",  #yellow
         -1:"#fdd835", #yellow
         -2:"#bcaaa4", #brown
         -3:"#ffd54f", #amber
         -4:"#ffc107", #amber
         -5:"#ff7043", #orange
         -6:"#f4511e", #orange
         -7:"#e64a19", #orange
         -8:"#ef5350", #red
         -9:"#f44336", #red
         -10:"#b71c1c", #red
         1:"#fff176",  #yellow
         2:"#bcaaa4", #brown
         3:"#ffd54f", #amber
         4:"#ffc107", #amber
         5:"#ff7043", #orange
         6:"#f4511e", #orange
         7:"#e64a19", #orange
         8:"#ef5350", #red
         9:"#f44336", #red
         10:"#b71c1c"}, #red
    # Winter
    "Winter" : {0:"#fafafa", 
         -1:"#b3e5fc", 
         -2:"#81d4fa", 
         -3:"#e3f2fd",
         -4:"#e3f2fd", 
         -5:"#bbdefb",
         -6:"#90caf9", 
         -7:"#64b5f6", 
         -8:"#42a5f5", 
         -9:"#2196f3", 
         -10:"#1e88e5", 
         1:"#b3e5fc",  
         2:"#81d4fa", 
         3:"#e3f2fd", 
         4:"#e3f2fd", 
         5:"#bbdefb", 
         6:"#90caf9", 
         7:"#64b5f6", 
         8:"#42a5f5", 
         9:"#2196f3", 
         10:"#1e88e5"}, 
    # Sage Green
    "SageGreen" : {0:"#fafafa", 
         -1:"#c8e6c9", 
         -2:"#f1f8e9", 
         -3:"#dcedc8",
         -4:"#dcedc8", 
         -5:"#c5e1a5",
         -6:"#aed581", 
         -7:"#9ccc65", 
         -8:"#33691e", 
         -9:"#33691e", 
         -10:"#689f38", 
         1:"#c8e6c9",  
         2:"#f1f8e9", 
         3:"#dcedc8", 
         4:"#dcedc8", 
         5:"#c5e1a5", 
         6:"#aed581", 
         7:"#9ccc65", 
         8:"#33691e", 
         9:"#33691e", 
         10:"#689f38"}, 
    # Reds
    "Reds" : {0:"#fafafa", 
         -1:"#ef9a9a", 
         -2:"#ef9a9a", 
         -3:"#ba68c8",
         -4:"#ba68c8", 
         -5:"#d81b60",
         -6:"#d81b60", 
         -7:"#d81b60", 
         -8:"#b71c1c", 
         -9:"#b71c1c", 
         -10:"#b71c1c", 
         1:"#ef9a9a",  
         2:"#ef9a9a", 
         3:"#ba68c8", 
         4:"#ba68c8", 
         5:"#d81b60", 
         6:"#d81b60", 
         7:"#d81b60", 
         8:"#b71c1c", 
         9:"#b71c1c", 
         10:"#b71c1c"}, 
    #blue/green
    "Blue/Green" : {0:"#fafafa", 
         -1:"#c8e6c9", 
         -2:"#f1f8e9", 
         -3:"#dcedc8",
         -4:"#dcedc8", 
         -5:"#c5e1a5",
         -6:"#aed581", 
         -7:"#9ccc65", 
         -8:"#33691e", 
         -9:"#33691e", 
         -10:"#689f38", 
         1:"#b3e5fc",  
         2:"#81d4fa", 
         3:"#0d47a1", 
         4:"#1976d2", 
         5:"#bbdefb", 
         6:"#90caf9", 
         7:"#64b5f6", 
         8:"#42a5f5", 
         9:"#2196f3", 
         10:"#1e88e5"}, 
    #Solana
    "Solana" : {0:"#00FFA3", 
         -1:"#00FFA3", 
         -2:"#03E1FF", 
         -3:"#03E1FF",
         -4:"#DC1FFF", 
         -5:"#00FFA3",
         -6:"#DC1FFF", 
         -7:"#03E1FF", 
         -8:"#03E1FF", 
         -9:"#DC1FFF", 
         -10:"#00FFA3", 
         1:"#00FFA3",  
         2:"#03E1FF", 
         3:"#03E1FF", 
         4:"#DC1FFF", 
         5:"#00FFA3", 
         6:"#DC1FFF", 
         7:"#03E1FF", 
         8:"#03E1FF", 
         9:"#DC1FFF", 
         10:"#00FFA3"}, 
    #LP
    "LP" : {0:"#E0E1DE", 
         -1:"#FF4D46", 
         -2:"#BCD7B9", 
         -3:"#A0BD49",
         -4:"#A0BD49", 
         -5:"#FB7D31",
         -6:"#FB7D31", 
         -7:"#FB7D31", 
         -8:"#FF4D46", 
         -9:"#FF4D46", 
         -10:"#FF4D46", 
         1:"#FB7D31",  
         2:"#FF4D46", 
         3:"#A0BD49", 
         4:"#A0BD49", 
         5:"#FB7D31", 
         6:"#FB7D31", 
         7:"#FB7D31", 
         8:"#FF4D46", 
         9:"#FF4D46", 
         10:"#FF4D46"}, 
}

backgrounds = {
    # black
    0 : (0, 0, 0),
    # white
    #1 : (255, 255, 255),
    # grey
    #2 : (192, 192, 192),
}

shapes = ["Circle", "Square", "Mixed"]

In [18]:
file_index = 0

for root, _, files in os.walk("wav_files"):
    for name in files:
        if os.path.join(root,name) == "wav_files/.DS_Store":
            continue

        print(name)
        
        # Ingest song as numpy array
        a = read(os.path.join(root,name))
        # Convert array to float
        a = np.array(a[1],dtype=float)
        # Concatenate subarrays, if needed
        if type(a[0]) == np.ndarray:
            a = np.concatenate(a).ravel()
            
        # Normalize
        a = a * scaling_factor/(np.amax(a))
        # Convert to int
        a = a.astype(int)
        
        # DEFINE PARAMETERS
        # Sparsity (percentage of zeroes)
        sparsity = np.random.rand()#len(a[a == 0])/len(a)
        print(sparsity)
        
        # Remove zeroes from array
        a = a[a != 0]
        
        # Intensity (mean/max)
        intensity = np.mean(abs(a))/np.max(abs(a))
        print(intensity)
        
        for palette in palettes.keys():
            for background in range(0, len(backgrounds)):
                for dot_scaling_factor in dot_scaling_factors:
                    for num_dots in nums_dots:
                        for shape in shapes:

                            im = Image.new('RGB', (width, height), color=backgrounds[background])

                            draw = ImageDraw.Draw(im)

                            for value in a[starting_index:starting_index+num_dots]:

                                topLeftX = np.random.rand()*width
                                topLeftY = np.random.rand()*height
                                botRightX = topLeftX + (width/dot_scaling_factor)*np.random.rand()/(sparsity)
                                botRightY = botRightX - topLeftX + topLeftY

                                if shape == "Square":
                                    draw.rectangle((topLeftX, topLeftY, botRightX, botRightY), palettes[palette][value])
                                elif shape == "Circle":
                                    draw.ellipse((topLeftX, topLeftY, botRightX, botRightY), palettes[palette][value])
                                elif shape == "Mixed":
                                    if np.random.rand() > 0.5:
                                        draw.rectangle((topLeftX, topLeftY, botRightX, botRightY), palettes[palette][value])
                                    else:
                                        draw.ellipse((topLeftX, topLeftY, botRightX, botRightY), palettes[palette][value])

                            # Remove .wav extension from file name before saving
                            name = name.split(".")[0]

                            # Initialize metadata json for each NFT
                            metadata = dict()

                            # Construct metadata
                            metadata= {
                                "name" : f"Synesthesia #{file_index}",
                                "symbol" : "",
                                "description" : f"Piece #{file_index} in the Synesthesia collection",
                                "seller_fee_basis_points" : 250,
                                "image" : f"{str(file_index)}.png",
                                "attributes": [{"trait_type": "Source Song",
                                                "value": f"{name.split('_')[1]}"},
                                               {"trait_type": "Source Artist",
                                                "value": f"{name.split('_')[0]}"},
                                               {"trait_type": "Dot Scaling Factor",
                                                "value": f"{dot_scaling_factor}"},
                                               {"trait_type": "Palette",
                                                "value": f"{palette}"},
                                              {"trait_type": "Shape",
                                                "value": f"{shape}"},
                                              {"trait_type": "Density",
                                                "value": f"{num_dots}"},],
                                "properties" : {
                                    "files": [{"uri": f"{str(file_index)}.png",
                                               "type": "image/png"}],
                                    "category":"image/png",
                                    "creators": [{"address" : "8ejegyXpDAgDvQ8poRnK21CHkJGBrsLxfxNMSFCoNiuA",
                                                  "share" : 100}],
                                }
                            }

                            # Save metadata into a json file
                            with open(f"output/{str(file_index)}.json", "w", encoding="utf-8") as outfile:
                                json.dump(metadata, outfile, ensure_ascii=False, indent=4)

                            # Save image
                            im.save(f"output/{file_index}.jpg", optimize=True)
                            os.rename(f"output/{file_index}.jpg",f"output/{file_index}.png")
                            file_index += 1

Mike Shinoda_Ziggurats.wav
0.5953168737476426
0.2935314875521207
